In [ ]:
import sys
import os
sys.path.append('..')
from mpp.python.mppy import mpp
from tools.seismogram_lib import read_seismograms, plot_seismograms, calculate_rel_difference
import re
from copy import deepcopy
import random

In [ ]:
mpp.build()

In [ ]:
a = mpp.run_get_output(64, ["marmousi2_ml"])

In [ ]:
print(a)

In [ ]:
def parse_output(output):
    result = {}
    result["seismograms"] = re.findall("write measure to (.*)\n", output)[-1].replace(".000000","").replace("00000_","_")
    result["duration"], result["procs"] = re.findall("end program after (.*) minutes on ([0-9]*) procs", output)[0]
    minutes, seconds = map(float,result["duration"].split(":"))
    result["time"] = int(minutes*60+seconds)
    
    
    
    keylen = max(map(len,result.keys()))
    
    #for key, val in result.items():
    #    print(key.ljust(keylen), ":", val)
    return result

In [ ]:
parse_output(a)

In [ ]:
class Variable:
    def __init__(self, name, value, bounds, step):
        self.name = name
        self.value = value
        self.bounds = bounds
        self.step = step
        self.history = [self.value]
        
    def decrease(self):       
        value = self.value
        if type(self.step) is int:
            value -= self.step
        if type(self.step) is float:
            value *= self.step      
        
        self.value = max(value, self.bounds[0])
        self.history.append(self.value)
        
    def randomize(self):
        if type(self.step) is int:
            self.value = random.randint(*self.bounds)
        if type(self.step) is float:
            self.value =  random.uniform(*self.bounds)      
    
    def __call__(self):
        return self.name+ "=" + str(self.value)
    
        
    def increase(self):
        value = self.value
        if type(self.step) is int:
            value += self.step
        if type(self.step) is float:
            value /= self.step       
        
        self.value = min(value, self.bounds[1])
        self.history.append(self.value)
    
    
    def hist_back(self):
        self.value = self.history.pop()
        
    
    
    def __str__(self):
        return "[{}, {}, {}, {}]".format(self.name, self.value, self.bounds, self.step)
    
    def __repr__(self):
        return str(self)
    
    def __eq__(self, other):
        return self.value == other.value and self.name == other.name
    
    

In [ ]:
#results = []
fields = [ Variable("SmootherDamp_space", 1,[0.5,1], 0.97), 
           Variable("Presmoothing_space", 1,[1,30], 3), 
           Variable("Postsmoothing_space", 1,[1,30], 3), 
           Variable("SmootherDamp_time", 1,[0.5,1], 0.97), 
           Variable("Presmoothing_time", 1,[0,30], 3),           
           Variable("Postsmoothing_time", 1,[0,30], 3)
         ]
         


best_time = 240
best_config = deepcopy(fields)

for i in range(200):
    
    #mutated = []
    #while not mutated:
    #    for f in fields:
    #        if random.random() < 1/6:
    #            mutated.append(f)                
    #for m in mutated:
    #    random.choice([f.increase, f.decrease])()
    
    #for f in fields:
    #    f.randomize()
    
    params = [v() for v in fields]
    
    a = mpp.run_get_output(64, ["marmousi2_ml"]+params)
    output = parse_output(a)
    results.append((params,output, a))
    
    time = output["time"]
    
    print(best_time, time, " ".join(v() for v in fields))
    
    if time < best_time:
        best_time = time
        best_config = deepcopy(fields)
    #else:
    #    for m in mutated:
    #        m.hist_back()
    
    

In [ ]:
results[1][0::2]

In [ ]:
s1 = "data/s_ST_l_2_probl_2_sDeg_2_tDeg_2_nL_0_pml_0.000000_dt_16_adapt_0_tauz_0.000000_taup_0.100000_kapaf_1.100000_dgInTime"
s2 = "data/s_ST_l_2_probl_2_sDeg_2_tDeg_2_nL_0_pml_0.000000_dt_16_adapt_0_tauz_0.000000_taup_0.100000_kapaf_1.050000_dgInTime"
files = [s1,s2]
files = ["../build/"+l for l in files]
seismograms,(d_min,d_max) = read_seismograms(files)
import matplotlib.pyplot as plt
plot_seismograms(seismograms, d_min, d_max,figsize=(20,20), xlim=(0,1.5))
calculate_rel_difference(seismograms)

#diff = seismograms[0]-seismograms[1]
#import numpy as np
#plot_seismograms([diff/np.max(np.abs(diff))],-1,1)